In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')
import seaborn as sns
import random

import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics
from sklearn.preprocessing import StandardScaler
import os

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/train.csv')
print(train.shape)
test = pd.read_csv('/kaggle/input/tabular-playground-series-feb-2022/test.csv')
print(test.shape)

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
train.head()

In [ ]:
features = [x for x in test.columns.values if x != 'row_id']

train = train[features]
test = test[features]

train['target'] = 0
test['target'] = 1

df = pd.concat([train, test], axis =0)

In [ ]:
del train, test

In [ ]:
param = {'num_leaves': 150,
         'objective':'binary',
         'max_depth': 5,
         'learning_rate': 0.08,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 2022,
         "metric": 'auc',
         "verbosity": -1}

In [ ]:
train, test = model_selection.train_test_split(df, test_size=0.3, random_state=2022, shuffle=True)

train_y = train['target'].values
test_y = test['target'].values

train = train.drop('target', axis = 1)
test = test.drop('target', axis = 1)

train = lgb.Dataset(train, label=train_y)
test = lgb.Dataset(test, label=test_y)

num_round = 300
clf = lgb.train(param, train, num_round, valid_sets = [train, test], verbose_eval=50, early_stopping_rounds = 50)

In [ ]:
feature_imp = pd.DataFrame(sorted(zip(clf.feature_importance(),features)), columns=['Value','Feature'])

plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False).head(20))
plt.title('LightGBM Features')
plt.tight_layout()
plt.show()